### ------------------------------------------------------------------------------------------------------------

In [ ]:
%pylab inline

In [ ]:
import os

if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/impulsecorp/PickStocks.git
    !mv PickStocks/*.py.
    !mv PickStocks/data.
    !pip install -qq -r PickStocks/requirements.txt
    !pip install -qq scikit-optimize

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import system
from system import *
from datablock import *
from backtesting import Backtest
from sklearn.linear_model import LogisticRegression
seed

### ------------------------------------------------------------------------------------------------------------

In [ ]:
%%time

data_timeperiod = '15min'
data = get_data('SPY', period=data_timeperiod)
data = procdata_lite(data, use_tsfel=0)

In [ ]:
# print(get_data_features(data))
data.shape

In [ ]:
system.train_set_end = 0.4 # percentage point specifying the training set end point (1.0 means all data is training set)
system.val_set_end = 0.7 # percentage point specifying the validation set end point (1.0 means no test set)
# basically this is the data with the values above, which are like sliders determining the layout
# [|0.0| ...... train ........ |0.4| ............ val ............ |0.7| .............. test ............... |1.0|]

### ------------------------------------------------------------------------------------------------------------

In [ ]:
# Let's find the best minimum confidence to enter a trade

In [ ]:
%%time

# search space boundaries
sp_min = 0.0
sp_max = 1.0
points = 1000

MLEnsembleStrategy.num_clfs = 20
MLEnsembleStrategy.dropout = 0.05

optkw['method'] = 'skopt'
optkw['max_tries'] = 50

bt = Backtest(data, MLEnsembleStrategy, **btkw)
consts = dict(mode=['opt'], clf_class=[LogisticRegression], period=[data_timeperiod])
results = bt.optimize(**consts, **optkw, random_state=reseed(),
                      min_confidence=np.linspace(sp_min, sp_max, points).tolist())
plot_optresult(get_optdata(results, consts), 'min_confidence');

In [ ]:
stats = results[0]
stats

In [ ]:
trades = stats['_trades']
trades.head()

In [ ]:
# performance of best backtest
plot_result(bt, results)

In [ ]:
gr = get_optdata(results, consts)
best_min_confidence = gr.index[np.argmax(gr.values)]
best_min_confidence, np.max(gr.values)

In [ ]:
%%time

# testing the optimized model on unseen data here
consts['mode'] = ['test']
stats = bt.run(**{x:consts[x][0] for x in consts.keys()},
               min_confidence=best_min_confidence)
display(stats)

In [ ]:
plot_result(bt, [stats])

### ------------------------------------------------------------------------------------------------------------

In [ ]:
# Here we freeze this best confidence and search for best ensemble size with it

In [ ]:
# search space boundaries
sp_min = 5
sp_max = 200
system.max_tries = 0.05

MLEnsembleStrategy.dropout = 0.05
MLEnsembleStrategy.min_confidence = best_min_confidence

optkw['method'] = 'skopt'
optkw['max_tries'] = 50

bt = Backtest(data, MLEnsembleStrategy, **btkw)
consts = dict(mode=['opt'], clf_class=[LogisticRegression], period=[data_timeperiod])
results = bt.optimize(**consts, **optkw, random_state=reseed(),
                      num_clfs=np.arange(sp_min, sp_max).tolist())
plot_optresult(get_optdata(results, consts), 'num_clfs');

In [ ]:
stats = results[0]
stats

In [ ]:
# performance of best backtest
plot_result(bt, results)

### ------------------------------------------------------------------------------------------------------------

In [ ]:
# Let's test that on unseen data

In [ ]:
gr = get_optdata(results, consts)
best_num_clfs = gr.index[np.argmax(gr.values)]
best_num_clfs, np.max(gr.values)

In [ ]:
%%time

# testing the optimized model on unseen data here
consts['mode'] = ['test']
stats = bt.run(**{x:consts[x][0] for x in consts.keys()},
               num_clfs=best_num_clfs)
display(stats)

In [ ]:
plot_result(bt, [stats])

### ------------------------------------------------------------------------------------------------------------